In [41]:
import numpy as np
import lda
import lda.datasets
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
print(X.shape,X.sum(), "vocab:", len(vocab), "titles:", len(titles))


(395, 4258) 84010 vocab: 4258 titles: 395


In [42]:
model = lda.LDA(n_topics=100, n_iter=1500, random_state=1)
model.fit(X)  # model.fit_transform(X) is also available
topic_word = model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 395
INFO:lda:vocab_size: 4258
INFO:lda:n_words: 84010
INFO:lda:n_topics: 100
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -1258788
INFO:lda:<10> log likelihood: -734487
INFO:lda:<20> log likelihood: -708068
INFO:lda:<30> log likelihood: -698147
INFO:lda:<40> log likelihood: -691363
INFO:lda:<50> log likelihood: -687491
INFO:lda:<60> log likelihood: -684165
INFO:lda:<70> log likelihood: -682336
INFO:lda:<80> log likelihood: -680400
INFO:lda:<90> log likelihood: -678905
INFO:lda:<100> log likelihood: -676797
INFO:lda:<110> log likelihood: -675647
INFO:lda:<120> log likelihood: -675180
INFO:lda:<130> log likelihood: -674108
INFO:lda:<140> log likelihood: -673157
INFO:lda:<150> log likelihood: -673066
INFO:lda:<160> log likelihood: -672008
INFO:lda:<170> log likelihood: -671173
INFO:lda:<180> log likelihood: -671111
INFO:lda:<190> log likelihood: -670026
INFO:lda:<200> log likelihood: -669592
INFO:lda:<210> log likelihood: -668796
INFO:lda:<220> log likelihood: 

Topic 0: buddhist vietnam cancer national friday local dent ceremony
Topic 1: christmas year state days day time church held
Topic 2: social party major leader leadership beetham credit since
Topic 3: people n't own years world added end right
Topic 4: political union including leader took leading battle soldiers
Topic 5: hwang ramos estrada philippines state manila north korean
Topic 6: million sale went estate estimated auction sotheby former
Topic 7: president ceremony former thursday house honour senate tsongas
Topic 8: wallace including city people crowd procession mayor hundreds
Topic 9: doctors home condition hospital sen prayers tuesday nursing
Topic 10: yeltsin kremlin operation president heart russian chernomyrdin russia
Topic 11: art century exhibition works show artists visitors great
Topic 12: church economic reforms independent labour leaders police law
Topic 13: first star n't singer madonna child appeared father
Topic 14: harriman u.s churchill ambassador paris pamela f

In [10]:
doc_topic = model.doc_topic_
for i in range(10):
    print("{} (top topic: {})".format(titles[i], doc_topic[i].argmax()))

0 UK: Prince Charles spearheads British royal revolution. LONDON 1996-08-20 (top topic: 66)
1 GERMANY: Historic Dresden church rising from WW2 ashes. DRESDEN, Germany 1996-08-21 (top topic: 83)
2 INDIA: Mother Teresa's condition said still unstable. CALCUTTA 1996-08-23 (top topic: 79)
3 UK: Palace warns British weekly over Charles pictures. LONDON 1996-08-25 (top topic: 29)
4 INDIA: Mother Teresa, slightly stronger, blesses nuns. CALCUTTA 1996-08-25 (top topic: 79)
5 INDIA: Mother Teresa's condition unchanged, thousands pray. CALCUTTA 1996-08-25 (top topic: 79)
6 INDIA: Mother Teresa shows signs of strength, blesses nuns. CALCUTTA 1996-08-26 (top topic: 79)
7 INDIA: Mother Teresa's condition improves, many pray. CALCUTTA, India 1996-08-25 (top topic: 79)
8 INDIA: Mother Teresa improves, nuns pray for "miracle". CALCUTTA 1996-08-26 (top topic: 79)
9 UK: Charles under fire over prospect of Queen Camilla. LONDON 1996-08-26 (top topic: 29)


In [193]:
import json
with open('ac2-combined-ideas.json', 'r') as f:
    ac1_dict = json.load(f)["@graph"]
    content_array = list(map(lambda idea: idea["content"], ac1_dict));

with open('patent_25k.csv', 'r') as f:
    title = []
    content = []
    for line in f:
        arr = line.split("\t")
        title += [arr[7]]
        content += [arr[6]]
        
print(len(content),len(title))
content_array += content
#with open('ac2-abstract-ideas.json', 'r') as f:
#    ac2_dict = json.load(f)["@graph"]
#content_array += list(map(lambda idea: idea["content"], ac2_dict));
#print(content_array)


25000 25000


In [194]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(stop_words="english", max_df=0.1)
print("nr_of_sprowords:",len(vectorizer.get_stop_words()))
X = vectorizer.fit_transform(content_array)
vocab = vectorizer.get_feature_names()
print("nr_ideas:",len(content_array),"vocab:",len(vocab), X.shape)

318
nr_ideas: 25419 vocab: 14669 (25419, 14669)


In [195]:
new_model = lda.LDA(n_topics=10, n_iter=1500, random_state=1)
new_model.fit(X)  # model.fit_transform(X) is also available
topic_word = new_model.topic_word_  # model.components_ also works
n_top_words = 8
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

INFO:lda:n_documents: 25419
INFO:lda:vocab_size: 14669
INFO:lda:n_words: 143803
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
/usr/local/lib/python3.7/site-packages/lda/utils.py:55: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if sparse and not np.issubdtype(doc_word.dtype, int):
INFO:lda:<0> log likelihood: -1752692
INFO:lda:<10> log likelihood: -1310237
INFO:lda:<20> log likelihood: -1263846
INFO:lda:<30> log likelihood: -1235489
INFO:lda:<40> log likelihood: -1219208
INFO:lda:<50> log likelihood: -1207595
INFO:lda:<60> log likelihood: -1201061
INFO:lda:<70> log likelihood: -1196938
INFO:lda:<80> log likelihood: -1193265
INFO:lda:<90> log likelihood: -1190321
INFO:lda:<100> log likelihood: -1188742
INFO:lda:<110> log likelihood: -1187260
INFO:lda:<120> log likelihood: -1185692
INFO:lda:<130> log likelihood: -1184506
INFO:lda:<140> log likelihood: -11834

Topic 0: control power vehicle engine electric circuit gas air
Topic 1: data methods systems based using content storage network
Topic 2: display semiconductor light manufacturing having electronic liquid thereof
Topic 3: methods systems devices fluid using related measuring medical
Topic 4: methods thereof use compositions treatment compounds treating uses
Topic 5: image processing medium information imaging storage control forming
Topic 6: communication wireless network mobile using transmission optical data
Topic 7: used technology vehicle use detection movement people control
Topic 8: process composition producing material using manufacturing battery metal
Topic 9: assembly vehicle having connector support structure machine tool


In [196]:
doc_topic = new_model.doc_topic_
for i in range(10):
    print(np.sort(doc_topic[i])[::-1][:3])
    arr = np.arange(len(doc_topic))[np.argsort(doc_topic[i])][::-1][:3]
    print("{} (top topic: {})".format(content_array[i], arr))

[0.51666667 0.18333333 0.18333333]
The technology can be used to monitor air craft. (top topic: [7 2 0])
[0.76764706 0.20882353 0.00294118]
the device could be used in a detective manner that is to say that it could be useful to predicting behavioral patterns of say criminal offenders, students, to perform research perhaps by tracking the physical lives of athletes or top achieving businessman or for companies to isolate expected behaviors of their employees and how they anticipate employees to move during a work day to establish fair and true standards (top topic: [7 1 9])
[0.575      0.21785714 0.14642857]
An application could be created for pet owners who have pets who have ran away from home. The application could be downloaded right onto a persons phone. the hand sized device could be put into a collar of a pet, or a chip like device under the skin. The system would be able to pin point a pets location to the direct coordinates.  (top topic: [7 1 6])
[0.5125 0.2625 0.1375]
The tec